# MovieLens recommendation tutorial
Today we are going to build on what we learned during the course, and build several types of recommender systems for the movielens dataset, you can download it right [here](https://grouplens.org/datasets/movielens/1m/).
MovieLens is a web site that helps people find movies to watch. It has hundreds of thousands of registered users. And we are going to use this dataset with 1M ratings from 6000 users on 4000 movies (it was released in 2003 so it is quite old, but there are newer versions of this dataset)

## Part 1: Data preprocessing
First we are going just to preprocess the data a bit to get it in a clear manner, and replace some of the column values by clearer values, nothing too fancy.

In [ ]:
# Import packages
import os
import pandas as pd
import random

random.seed(42) 

In [ ]:
# Define directories
MOVIELENS_DIR = 'ml-1m'
USER_DATA_FILE = 'users.dat'
MOVIE_DATA_FILE = 'movies.dat'
RATING_DATA_FILE = 'ratings.dat'

In [ ]:
# Specify User's Age and Occupation Column
AGES = { 1: "Under 18", 18: "18-24", 25: "25-34", 35: "35-44", 45: "45-49", 50: "50-55", 56: "56+" }
OCCUPATIONS = { 0: "other or not specified", 1: "academic/educator", 2: "artist", 3: "clerical/admin",
                4: "college/grad student", 5: "customer service", 6: "doctor/health care",
                7: "executive/managerial", 8: "farmer", 9: "homemaker", 10: "K-12 student", 11: "lawyer",
                12: "programmer", 13: "retired", 14: "sales/marketing", 15: "scientist", 16: "self-employed",
                17: "technician/engineer", 18: "tradesman/craftsman", 19: "unemployed", 20: "writer" }

In [ ]:
# Define csv files to be saved into
USERS_CSV_FILE = 'users.csv'
MOVIES_CSV_FILE = 'movies.csv'
RATINGS_CSV_FILE = 'ratings.csv'

In [ ]:
# Read the Ratings File
ratings = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Set max_userid to the maximum user_id in the ratings
max_userid = ratings['user_id'].drop_duplicates().max()
# Set max_movieid to the maximum movie_id in the ratings
max_movieid = ratings['movie_id'].drop_duplicates().max()

print(len(ratings), 'ratings loaded')

In [ ]:
# Let's just check what it looks like
ratings.head()

In [ ]:
# Save into ratings.csv
ratings.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True,
               index=False,
               encoding='latin-1')
print('Saved to', RATINGS_CSV_FILE)

In [ ]:
# Read the Users File
users = pd.read_csv(os.path.join(MOVIELENS_DIR, USER_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['user_id', 'gender', 'age', 'occupation', 'zipcode'])
users['age_desc'] = users['age'].apply(lambda x: AGES[x])
users['occ_desc'] = users['occupation'].apply(lambda x: OCCUPATIONS[x])

print(len(users), 'descriptions of', max_userid, 'users loaded.')

In [ ]:
users.head()

In [ ]:
# Save into users.csv
users.to_csv(USERS_CSV_FILE, 
             sep='\t', 
             header=True,
             index=False,
             encoding='latin-1')
print('Saved to', USERS_CSV_FILE)

In [ ]:
# Read the Movies File
movies = pd.read_csv(os.path.join(MOVIELENS_DIR, MOVIE_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movie_id', 'title', 'genres'])
print(len(movies), 'descriptions of', max_movieid, 'movies loaded.')

In [ ]:
movies.head()

In [ ]:
# Save into movies.csv
movies.to_csv(MOVIES_CSV_FILE, 
              sep='\t', 
              header=True,
              index=False)
print('Saved to', MOVIES_CSV_FILE)

## Part 2: Movie Recommendation with Content-Based and Collaborative Filtering
“*What movie should I watch this evening?*” 

Have you ever had to answer this question at least once when you came home from work? As for me — yes, and more than once. From Netflix to Hulu, the need to build robust movie recommendation systems is extremely important given the huge demand for personalized content of modern consumers.

An example of recommendation system is such as this:
* User A watches **Game of Thrones** and **Breaking Bad**.
* User B does search on **Game of Thrones**, then the system suggests **Breaking Bad** from data collected about user A.

Recommendation systems are used not only for movies, but on multiple other products and services like Amazon (Books, Items), Pandora/Spotify (Music), Google (News, Search), YouTube (Videos) etc.

Two most ubiquitous types of personalized recommendation systems are **Content-Based** and **Collaborative Filtering**. Collaborative filtering produces recommendations based on the knowledge of users’ attitude to items, that is it uses the “wisdom of the crowd” to recommend items. In contrast, content-based recommendation systems focus on the attributes of the items and give you recommendations based on the similarity between them.

In this notebook, we will attempt at implementing these two systems to recommend movies and evaluate them to see which one performs better.

### Data preparation
Let's load this data into Python. we will load the dataset with Pandas onto Dataframes **ratings**, **users**, and **movies**.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1')

# Reading users file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1')

# Reading movies file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1')

Now lets take a peak into the content of each file to understand them better.

### Ratings Dataset

In [ ]:
ratings.head()

In [ ]:
ratings.info()

This confirms that there are approximately 1M ratings for different user and movie combinations.

### Users Dataset

In [ ]:
users.head()

In [ ]:
users.info()

This confirms that there are 6040 users and we have 5 features for each (unique user ID, gender, age, occupation and the zip code they are living in).

### Movies Dataset

In [ ]:
movies.head()

In [ ]:
movies.info()

This dataset contains attributes of the 3883 movies. There are 3 columns including the movie ID, their titles, and their genres. Genres are pipe-separated and are selected from 18 genres (Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western).

## Data Exploration
### Titles
Are there certain words that feature more often in Movie Titles? Let's try out a nice visualization using word clouds.

In [ ]:
%matplotlib inline

import wordcloud
from wordcloud import WordCloud, STOPWORDS

# Create a wordcloud of the movie titles
movies['title'] = movies['title'].fillna("").astype('str')
# we just join all the titles together into a big string
title_corpus = ' '.join(movies['title'])

title_wordcloud = WordCloud(stopwords=STOPWORDS, background_color='black', height=2000, width=4000).generate(title_corpus)

# Plot the wordcloud
plt.figure(figsize=(16,8))
plt.imshow(title_wordcloud)
plt.axis('off')
plt.show()

Ok maybe not the most interesting visualization in the world, but gives us an idea of what words are in the movie titles. We can see Day, Man, Love ...

### Ratings
Now let's examine the ratings a bit more. Some descriptive statistics first.

In [ ]:
ratings['rating'].describe()

It appears that users are quite generous in their ratings. The mean rating is 3.58 on a scale of 5. Half the movies have a rating of 4 and 5. But because different people might rate on different scales, we can't really conclude.

Let's also take a look at a subset of 20 movies with the highest rating.

In [ ]:
# Join all 3 files into one dataframe
dataset = pd.merge(pd.merge(movies, ratings),users)

# Display 20 movies with highest ratings
dataset[['title','genres','rating']].sort_values('rating', ascending=False).head(10)

### Genres
The genres variable will surely be important while building the recommendation engines since it describes the content of the film (i.e. Animation, Horror, Sci-Fi). A basic assumption is that films in the same genre should have similar contents.

In [ ]:
from collections import Counter

genre_counter = Counter([genre for row in movies['genres'].apply(lambda s: s.split('|')).values for genre in row])

In [ ]:
genre_counter.most_common(5)

The top 5 genres are, in that respect order: Drama, Comedy, Action, Thriller, and Romance. Let's visualize them in a wordcloud again.

In [ ]:
# Create the wordcloud
genre_wordcloud = WordCloud(width=1000,height=400, background_color='white')
genre_wordcloud.generate_from_frequencies(genre_counter)

# Plot the wordcloud
f, ax = plt.subplots(figsize=(16, 8))
plt.imshow(genre_wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

### Content based collaborative filtering
Like we said in the course, the metadata attached to an item can be used to compute similarities, and recommend to a user similar items to what he saw.
We saw briefly how can we use TF-IDF to compute item vectors from descriptions, categories, ... and then use the similarity between item vectors (cosine similarity for instance).
We are going to do that here with movies genres.

In [ ]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
# Convert genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', min_df=0)
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim[:4, :4]

We now have a pairwise cosine similarity matrix for all the movies in the dataset. The next step is to write a function that returns the most similar movies based on the cosine similarity score.

In [ ]:
# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def recommendations_from_similarities(sims, title, k=10):
    idx = indices[title]
    sim_scores = list(enumerate(sims[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:k+1]
    # the most similar movie to a given one will always be itself, so we skip that one
    movie_indices = [i[0] for i in sim_scores if i[0] != idx]
    return titles.iloc[movie_indices]

In [ ]:
recommendations_from_similarities(cosine_sim, 'Matrix, The (1999)').head(10)

In [ ]:
recommendations_from_similarities(cosine_sim, 'Toy Story (1995)').head(10)

In [ ]:
recommendations_from_similarities(cosine_sim, 'Saving Private Ryan (1998)').head(10)

### Collaborative Filtering
Now we are going to apply what we learned about item-item and user-user collaborative filtering.

First we need to split our data to test and train datasets, so we can evaluate what we are doing.

In [ ]:
# Fill NaN values in user_id and movie_id column with 0
ratings['user_id'] = ratings['user_id'].fillna(0)
ratings['movie_id'] = ratings['movie_id'].fillna(0)

# Replace NaN values in rating column with average of all values
ratings['rating'] = ratings['rating'].fillna(ratings['rating'].mean())

In [ ]:
# split into 80% train and 20% test
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(ratings, test_size=0.2)

In [ ]:
import scipy

def to_sparse_matrix(df, rating_col="rating"):
    return scipy.sparse.csr_matrix((df[rating_col], (df['user_id'], df['movie_id'])))

In [ ]:
train_sparse = to_sparse_matrix(train_data)

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_sim = 1 - pairwise_distances(train_sparse, metric='cosine')
user_sim[np.isnan(user_sim)] = 0

print(user_sim[:4, :4])

In [ ]:
# Item Similarity Matrix

item_sim = 1 - pairwise_distances(train_sparse.T, metric='cosine')
item_sim[np.isnan(item_sim)] = 0

print(item_sim[:4, :4])

In [ ]:
recommendations_from_similarities(item_sim, 'Matrix, The (1999)').head(10)

In [ ]:
recommendations_from_similarities(item_sim, 'Toy Story (1995)').head(10)

In [ ]:
recommendations_from_similarities(item_sim, 'Saving Private Ryan (1998)').head(10)

In [ ]:
# Adjusted cosine similarity
# We need to substract average rating for each item from its rating
train_data_adj = train_data.join(train_data.groupby('movie_id')['rating'].mean(), on='movie_id', rsuffix='_mean')

In [ ]:
train_data_adj["adjusted_rating"] = train_data_adj["rating"] - train_data_adj["rating_mean"]

In [ ]:
train_data_adj.head()

In [ ]:
train_adj_sparse = to_sparse_matrix(train_data_adj, rating_col="adjusted_rating")

In [ ]:
# Item Similarity Matrix

item_sim_adj = 1 - pairwise_distances(train_adj_sparse.T, metric='cosine')
item_sim_adj[np.isnan(item_sim_adj)] = 0

print(item_sim_adj[:4, :4])

In [ ]:
recommendations_from_similarities(item_sim_adj, 'Saving Private Ryan (1998)').head(10)

### Writing the prediction functions
Now we are going to write a couple of functions that will predict ratings for user, item pairs in the test set.
We will be using the formulas from the course, to take the weighted average of ratings from:
* The items that the user already rated, weighted by similarity to the target item
* The ratings that the other users gave to the item, weighted by the similarity with the target user

In [ ]:
from scipy import sparse

def predict_user_user_cf(train_ratings, test_ratings, similarity_matrix):
    mean_user_ratings = train_data.groupby('user_id')['rating'].mean()
    
    train_ratings = train_ratings.join(mean_user_ratings, on='user_id', rsuffix='_user_mean')
    test_ratings = test_ratings.join(mean_user_ratings, on='user_id', rsuffix='_user_mean')
    
    train_ratings["ratings_adjusted_user_mean"] = train_ratings["rating"] - train_ratings["rating_user_mean"]
    train_ratings_sparse = to_sparse_matrix(train_ratings, "ratings_adjusted_user_mean")
    
    ratings_weighted_by_user_sim = sparse.csr_matrix.dot(similarity_matrix, train_ratings_sparse) / np.sum(similarity_matrix, axis=1, keepdims=True)
    
    indices = np.array([[user_id, movie_id] for user_id in range(ratings_weighted_by_user_sim.shape[0]) for movie_id in range(ratings_weighted_by_user_sim.shape[1])])
    predicted_ratings = ratings_weighted_by_user_sim.flatten()
    
    predicted_ratings_df = pd.DataFrame({
        "user_id": indices[:, 0],
        "movie_id": indices[:, 1],
        "predicted_rating": predicted_ratings,
    })
    
    predicted_ratings_df = pd.merge(predicted_ratings_df, test_ratings, on=["user_id", "movie_id"])
    predicted_ratings_df["predicted_rating"] = predicted_ratings_df["predicted_rating"] + predicted_ratings_df["rating_user_mean"]
    return predicted_ratings_df

In [ ]:
user_user_predictions = predict_user_user_cf(train_data, test_data, user_sim)
user_user_predictions.head()

In [ ]:
def predict_item_item_cf(train_ratings, test_ratings, similarity_matrix):
    mean_item_ratings = train_data.groupby('movie_id')['rating'].mean()
    
    train_ratings = train_ratings.join(mean_item_ratings, on='movie_id', rsuffix='_item_mean')
    test_ratings = test_ratings.join(mean_item_ratings, on='movie_id', rsuffix='_item_mean')
    
    train_ratings["ratings_adjusted_item_mean"] = train_ratings["rating"] - train_ratings["rating_item_mean"]
    train_ratings_sparse = to_sparse_matrix(train_ratings, "ratings_adjusted_item_mean")
    
    ratings_weighted_by_item_sim = sparse.csr_matrix.dot(train_ratings_sparse, similarity_matrix) / np.sum(similarity_matrix, axis=0, keepdims=True)
    
    indices = np.array([[user_id, movie_id] for user_id in range(ratings_weighted_by_item_sim.shape[0]) for movie_id in range(ratings_weighted_by_item_sim.shape[1])])
    predicted_ratings = ratings_weighted_by_item_sim.flatten()
    
    predicted_ratings_df = pd.DataFrame({
        "user_id": indices[:, 0],
        "movie_id": indices[:, 1],
        "predicted_rating": predicted_ratings,
    })
    
    predicted_ratings_df = pd.merge(predicted_ratings_df, test_ratings, on=["user_id", "movie_id"])
    predicted_ratings_df["predicted_rating"] = predicted_ratings_df["predicted_rating"] + predicted_ratings_df["rating_item_mean"]
    return predicted_ratings_df

In [ ]:
item_item_predictions = predict_item_item_cf(train_data, test_data, item_sim)

We are just going to replace nans here by the average predictions.

In [ ]:
item_item_predictions['predicted_rating'] = item_item_predictions['predicted_rating'].fillna(item_item_predictions['predicted_rating'].mean())

### Evaluation
We are going to evaluate our methods on the test set, using the mean absolute error metric.
Which is as follows:
$$\mathit{MAE} =\sqrt{\frac{1}{N} \sum |x_i -\hat{x_i}|}$$

This just measures the average deviation from the ground truth.

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print("MAPE of user-user cf", mean_absolute_error(user_user_predictions["rating"], user_user_predictions["predicted_rating"]))
print("MAPE of item-item cf", mean_absolute_error(item_item_predictions["rating"], item_item_predictions["predicted_rating"]))

We see here that item-item performs a bit better than user-user, at least on this dataset.
But to have a frame of reference, let's see what is the performance of a dumb predictor that just predicts the average rating

In [ ]:
test_data["average_rating"] = train_data["rating"].mean()
print("MAPE of average", mean_absolute_error(test_data["rating"], test_data["average_rating"]))

So we are indeed better than the baseline !

## Matrix Factorization
Now we are going to try our hand with matrix factorization techniques, we are going to use the **surprise** package which just bundles together a bunch of algorithms in a easy to use manner, check it out [here](https://surprise.readthedocs.io/en/stable/getting_started.html)

First we are going to build the dataset:

In [ ]:
from surprise import Dataset
from surprise import Reader
import surprise

In [ ]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

trainset, testset = surprise.model_selection.train_test_split(data, test_size=.2)

Then we are going to use the SVD algorithm, and compute the MAE on the test data, here the train - test split is not the same as previously (for ease of use with the surprise package) but it still gives us an idea of how we perform.

In [ ]:
# We'll use the famous SVD algorithm.
algo = SVD(n_factors=10, n_epochs=30, verbose=True)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

Let's compute the MAE

In [ ]:
from surprise import accuracy

accuracy.mae(predictions)

Seems it's better than our previous approaches ! Let's inspect the factors that we learned

In [ ]:
item_factors = algo.qi
item_factors_sim = 1 - pairwise_distances(item_factors, metric='cosine')

In [ ]:
item_factors_sim.shape

We need to redefine our function **recommendations_from_similarities** because of the specifics of the suprise package.

In [ ]:
id_to_title = dict((row["movie_id"], row["title"]) for _, row in movies.iterrows())
title_to_id = dict((row["title"], row["movie_id"]) for _, row in movies.iterrows())

# Function that get movie recommendations based on the cosine similarity score of movie genres
def recommendations_from_similarities_surprise(dataset, sims, title, k=10):
    movie_id = title_to_id[title]
    idx = dataset.to_inner_iid(movie_id)
    sim_scores = list(enumerate(sims[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:k+1]
    # the most similar movie to a given one will always be itself, so we skip that one
    movies = [id_to_title[dataset.to_raw_iid(i[0])] for i in sim_scores if i[0] != idx]
    return movies

In [ ]:
recommendations_from_similarities_surprise(trainset, item_factors_sim, 'Matrix, The (1999)')

In [ ]:
recommendations_from_similarities_surprise(trainset, item_factors_sim, 'Toy Story (1995)')

In [ ]:
recommendations_from_similarities_surprise(trainset, item_factors_sim, 'Saving Private Ryan (1998)')